![image.png](http://drive.google.com/uc?export=view&id=1MqoHuPys35fvTEu1Cv0KQ6Y3eoYGiQb0)

In [13]:
# 모델 참고 코드 
# https://github.com/wkentaro/pytorch-fcn/
import torch
import torch.nn as nn
class FCN8s(nn.Module):
    def __init__(self, num_classes=12):
        super(FCN8s, self).__init__()
        
        def CBR(in_channels, out_channels, kernel_size=3, stride=1, padding=1):
            return nn.Sequential(nn.Conv2d(in_channels=in_channels, 
                                            out_channels=out_channels,
                                            kernel_size=kernel_size,
                                            stride=stride, 
                                            padding=padding),
                                  nn.ReLU(inplace=True)
                                 )        
        #512x512
        # conv1
        self.conv1_1 = CBR(3, 64, 3, 1, 1) # 512-3+2+1=512
        self.conv1_2 = CBR(64, 64, 3, 1, 1) # 512-3+2+1=512
        self.pool1 = nn.MaxPool2d(2, stride=2, ceil_mode=True) # 256
        
        # conv2
        self.conv2_1 = CBR(64, 128, 3, 1, 1) # 256
        self.conv2_2 = CBR(128, 128, 3, 1, 1) # 256
        self.pool2 = nn.MaxPool2d(2, stride=2, ceil_mode=True) # 128

        # conv3
        self.conv3_1 = CBR(128, 256, 3, 1, 1) # 128
        self.conv3_2 = CBR(256, 256, 3, 1, 1) # 128
        self.conv3_3 = CBR(256, 256, 3, 1, 1) # 128
        self.pool3 = nn.MaxPool2d(2, stride=2, ceil_mode=True) # 64

        # Score pool3
        self.score_pool3_fr = nn.Conv2d(256,
                                        num_classes, 
                                        kernel_size=1,
                                        stride=1,
                                        padding=0)
        
        # conv4
        self.conv4_1 = CBR(256, 512, 3, 1, 1) # 64
        self.conv4_2 = CBR(512, 512, 3, 1, 1) # 64
        self.conv4_3 = CBR(512, 512, 3, 1, 1) # 64     
        self.pool4 = nn.MaxPool2d(2, stride=2, ceil_mode=True) # 32

        # Score pool4
        self.score_pool4_fr = nn.Conv2d(512,
                                        num_classes, 
                                        kernel_size=1,
                                        stride=1,
                                        padding=0)        
        
        # conv5
        self.conv5_1 = CBR(512, 512, 3, 1, 1) # 32
        self.conv5_2 = CBR(512, 512, 3, 1, 1) # 32
        self.conv5_3 = CBR(512, 512, 3, 1, 1) # 32
        self.pool5 = nn.MaxPool2d(2, stride=2, ceil_mode=True) # 16
    
        # fc6
        self.fc6 = nn.Conv2d(512, 4096, 1) # 4
        self.relu6 = nn.ReLU(inplace=True)
        self.drop6 = nn.Dropout2d()

        # fc7
        self.fc7 = nn.Conv2d(4096, 4096, 1) # 4
        self.relu7 = nn.ReLU(inplace=True)
        self.drop7 = nn.Dropout2d()


        # Score
        self.score_fr = nn.Conv2d(4096, num_classes, kernel_size = 1) # 4
        
        
        # UpScore2 using deconv
        self.upscore2 = nn.ConvTranspose2d(num_classes,
                                           num_classes,
                                           kernel_size=4,
                                           stride=2,
                                           padding=1) # x2 = 32
        
        # UpScore2_pool4 using deconv
        self.upscore2_pool4 = nn.ConvTranspose2d(num_classes, 
                                                 num_classes, 
                                                 kernel_size=4,
                                                 stride=2,
                                                 padding=1) # x2 = 64
        
        # UpScore8 using deconv
        self.upscore8 = nn.ConvTranspose2d(num_classes, 
                                           num_classes,
                                           kernel_size=16,
                                           stride=8,
                                           padding=4) # x8 = 512


    def forward(self, x):
        h = self.conv1_1(x) # 512
        h = self.conv1_2(h) # 512
        h = self.pool1(h) # 256

        h = self.conv2_1(h) # 256
        h = self.conv2_2(h) # 256
        h = self.pool2(h) # 128

        h = self.conv3_1(h) # 128
        h = self.conv3_2(h) # 128
        h = self.conv3_3(h) # 128
        pool3 = h = self.pool3(h) # 64

        # Score
        score_pool3c = self.score_pool3_fr(pool3) # channel = 256

        h = self.conv4_1(h) # 64
        h = self.conv4_2(h) # 64
        h = self.conv4_3(h) # 64 
        pool4 = h = self.pool4(h) # 32

        # Score
        score_pool4c = self.score_pool4_fr(pool4) # channel = 512
        
        h = self.conv5_1(h) # 32
        h = self.conv5_2(h) # 32
        h = self.conv5_3(h) # 32
        h = self.pool5(h) # 16
        
        h = self.fc6(h) # channel = 4096
        h = self.drop6(h)

        h = self.fc7(h) # channel = 4096
        h = self.drop7(h)
        
        h = self.score_fr(h) # channel = num_classes
        
        # Up Score I
        upscore2 = self.upscore2(h) # 32
        
        # Sum I
        h = upscore2 + score_pool4c
        
        # Up Score II
        upscore2_pool4c = self.upscore2_pool4(h) # 64
        
        # Sum II
        h = upscore2_pool4c + score_pool3c
        
        # Up Score III
        upscore8 = self.upscore8(h) # 512
        
        return upscore8